# 1_animal_classification.py

### Tensorflow, pandas, train_test_split 라이브러리 추가

In [24]:
#Tensorflow 추가
import tensorflow as tf

#pandas추가
import pandas as pd 

#sklearn의 train_test_split 라이브러리 추가(Dataset 내에서 train data와 test data를 분리하기 위해 사용)
from sklearn.model_selection import train_test_split 

### Data Set 불러오기

In [25]:
#csv 형식의 파일 read
zoo = pd.read_csv("./data/zoo.csv") 

### Data set의 열을 분리(Slice)

In [26]:
#zoo.csv의 마지막 열(column)인 class_type은 제외하고 animal_data 변수에 저장
animal_data = zoo.iloc[:,:-1] 

print(animal_data.head())

#zoo.csv의 마지막 열(column)인 class_type만 label_data 변수에 저장
label_data = zoo.iloc[:,-1:] 

print(label_data.head())

  animal_name  hair  feathers  eggs  milk  airborne  aquatic  predator  \
0    aardvark     1         0     0     1         0        0         1   
1    antelope     1         0     0     1         0        0         0   
2        bass     0         0     1     0         0        1         1   
3        bear     1         0     0     1         0        0         1   
4        boar     1         0     0     1         0        0         1   

   toothed  backbone  breathes  venomous  fins  legs  tail  domestic  catsize  
0        1         1         1         0     0     4     0         0        1  
1        1         1         1         0     0     4     1         0        1  
2        1         1         0         0     1     0     1         0        0  
3        1         1         1         0     0     4     0         0        1  
4        1         1         1         0     0     4     1         0        1  
   class_type
0           1
1           1
2           4
3           1
4    

### 전체 data set중 30%를 test데이터, 70%는 학습에 사용하기 위해 임의로 분할

In [27]:
train_x, test_x, train_y, test_y = train_test_split(animal_data,label_data,test_size=0.3,random_state=42,stratify=label_data)

첫번째 열인 동물이름(animal_name) 열을 분리

In [28]:
#train_x, test_x의 가장 첫 번째 animal_name 열을 train_name, test_name 변수에 저장
train_name = train_x['animal_name']
test_name = test_x['animal_name']

### train_x, test_x의 가장 첫 번째 동물이름(animal_name) 열을 제외하고 숫자 값만 갖는 데이터 형식으로 열을 분리(Slice) 

In [29]:
train_x = train_x.iloc[:,1:]
test_x = test_x.iloc[:,1:]

### tensorflow 라이브러리를 이용하여 네트워크 만들기
#### Input Layer 노드의 수 : 16개
#### 1번째 Hidden Layer 노드의 수 : 1개
#### Output Layer 노드의 수 : 1개

In [30]:
#Input Layer
#shape=(None, 7) : 입력하는 개수가 많을때는 정확히 알 수 없음(None), 한 번에 입력할 때 들어가는 데이터의 특징의 수 : 픽셀 9칸 -> 9
#Input Layer 노드의 수 = 16
X = tf.placeholder(dtype=tf.float32, shape=[None,16],name="X") #Input Layer 노드의 수 : 16

#Output Layer
Y = tf.placeholder(dtype=tf.int32, shape=[None,1], name="Y") #Output Layer 노드의 수 : 1

#Output Layer 노드의 수가 1개로 0인지 1인지 구별하기 위해 사용하는 방법이 One-hot encoding
# 동물원의 동물 분류 7종 분류에 대한 One-hot encoding
Y_one_hot = tf.one_hot(Y, 7)  # one hot encoding으로 7개 Class로 분류
Y_one_hot = tf.reshape(Y_one_hot, [-1, 7])

### 가중치(Weight : w)와 편향(Bias : b)

In [31]:
W = tf.Variable(tf.random_normal([16, 7],seed=0), name='W')

b = tf.Variable(tf.random_normal([7],seed=0), name='b')

### Input Layer 입력 값에 대한 가중치와 편향 계산 값

In [32]:
logit = tf.matmul(X,W)+b 

In [33]:
#입력데이터와 출력 데이터의 관계(Relationship) 또는 패턴(Pattern)을 나타내기 위한 함수 : hypothesis
hypothesis = tf.nn.softmax(logit)

In [34]:
#Logits를 통해 그려진 그래프와, hypothesis를 통해 판별한 결과의 오차를 계산
cost_i = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit,labels=Y_one_hot)

#전체 오차의 평균
cost = tf.reduce_mean(cost_i) 

In [35]:
#경사하강법(Gradient-Descent)를 이용하여 학습
#학습률(Learning_rate) : 0.05 -> 학습을 하는 과정에서 경사를 하강하는 폭의 정도 -> 작을 수록 폭이 좁음, 넓을 수록 폭이 넓음
train  = tf.train.GradientDescentOptimizer(learning_rate=0.05).minimize(cost)

In [36]:
#우리가 선정한 hypothesis 함수를 기반으로 Classification을 예측한 결과
prediction = tf.argmax(hypothesis, 1) 

#Prediction의 결과가 실제 Label 값과 맞는지 여부
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1)) 

#Prediction의 정확성을 저장하는 변수
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) 

### Tensorflow에서 값을 처리하기 위해 필요한 Session을 활용하여 실행

In [37]:
# Session 유형의 변수 sess 생성
with tf.Session() as sess:
    # Tensorflow 변수 초기화
    sess.run(tf.global_variables_initializer())
    # 5001번 반복하며 학습
    for step in range(5001):
        sess.run(train, feed_dict={X: train_x, Y: train_y})
        if step % 1000 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={X: train_x, Y: train_y})
            print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step, loss, acc))
    
    # 설계한 모델의 학습 정확도 계산
    train_acc = sess.run(accuracy, feed_dict={X: train_x, Y: train_y})
    print("Model Prediction =", train_acc)
    
    # 학습한 모델에 test_x, test_y을 이용하여 테스트 
    test_acc,test_predict,test_correct = sess.run([accuracy,prediction,correct_prediction], feed_dict={X: test_x, Y: test_y})
    print("Test Prediction =", test_acc)

Step:     0	Loss: 3.402	Acc: 30.00%
Step:  1000	Loss: 0.135	Acc: 87.14%
Step:  2000	Loss: 0.076	Acc: 90.00%
Step:  3000	Loss: 0.054	Acc: 90.00%
Step:  4000	Loss: 0.043	Acc: 90.00%
Step:  5000	Loss: 0.037	Acc: 90.00%
Model Prediction = 0.9
Test Prediction = 0.9354839


### 테스트 한 결과 출력

In [38]:
sub = pd.DataFrame()
sub['Name'] = test_name
sub['Predict_Type'] = test_predict
sub['Origin_Type'] = test_y
sub['Correct'] = test_correct

print(sub)

         Name  Predict_Type  Origin_Type  Correct
100      wren             2            2     True
58    penguin             2            2     True
43       lark             2            2     True
21       duck             2            2     True
10    cheetah             1            1     True
40   housefly             6            6     True
50   mongoose             1            1     True
4        boar             1            1     True
87       swan             2            2     True
80   slowworm             3            3     True
70   reindeer             1            1     True
37       hawk             2            2     True
7        carp             4            4     True
44    leopard             1            1     True
53    octopus             0            7     True
60       pike             4            4     True
84   squirrel             1            1     True
47       lynx             1            1     True
94       vole             1            1     True
